GOAL: Download and featurize DUDE dataset to predict bioactivity in molecules. i thought this will work at all. later I want to integrate spatial information between ligand and target, but how? binding poses might be found for actives but what about decoys? AutoDock might be able to help here, althought I do not quite understand if the predicted poses are actually possible posses (ATOMNet certaindly does something similar, althought it's not clear what)

First the test run with a single target, load data with rdkit, featurize everything with some fingerprint function and create a pandas data set, then we'll feed that into some very basic TF stuff I'll put together.